# Udacity Data Wrangling Project

### Importing Libraries and Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import tweepy
#from tweepy import OAuthHandler
#import json
#from timeit import default_timer as timer
import re
import requests


import twitter_credentials
%matplotlib inline

# Gather

###  Gathering Get Request

In [2]:
#response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
#if response.status_code == 200:
#    print('Success!')
#elif response.status_code == 404:
#    print('Not Found.')

In [3]:
images = pd.read_csv('image_predictions.tsv', sep = '\t', encoding = 'utf-8')

### Gathering data manually with Pandas

In [4]:
df_1 = pd.read_csv('twitter_archive_enhanced.csv')

### Gathering with the Twitter API



With a quick assessment of the tweet_id column I saw a unique problem I would need to solve first before being able to gather my last dataset containing all of the twitter data.

In [5]:
df_1.tweet_id.head()

0    8.924210e+17
1    8.921770e+17
2    8.918150e+17
3    8.916900e+17
4    8.913280e+17
Name: tweet_id, dtype: float64

Gathering tweets from the twitter API requires precise id's and unfortunately my csv was imported with a column of **twitter_id** that my unique id had suffered some rounding errors.

In [6]:
with pd.option_context('float_format', '{:.0f}'.format):  
    print(df_1.tweet_id.head())

0   892421000000000000
1   892177000000000000
2   891815000000000000
3   891690000000000000
4   891328000000000000
Name: tweet_id, dtype: float64


In [7]:
with pd.option_context('float_format', '{:.8f}'.format):  
    print(df_1.tweet_id.head())

0   892421000000000000.00000000
1   892177000000000000.00000000
2   891815000000000000.00000000
3   891690000000000000.00000000
4   891328000000000000.00000000
Name: tweet_id, dtype: float64


This gave me an oportunity to begin some data wrangling before jumping into the rest of my data. My solution to this issue was to split the strings of the URLs addresses that were included in the twitter_enhanced_archive csv file.  Using regular expression, I wanted to extract the Tweet Id's from the end of every URL. This required seperating the digits(these would be the id's I needed) from all of the char values that came before them, using '/' as a seperator.

In [8]:
df_1.expanded_urls.head()

0    https://twitter.com/dog_rates/status/892420643...
1    https://twitter.com/dog_rates/status/892177421...
2    https://twitter.com/dog_rates/status/891815181...
3    https://twitter.com/dog_rates/status/891689557...
4    https://twitter.com/dog_rates/status/891327558...
Name: expanded_urls, dtype: object


My first step was to drop and null value in  **expanded_urls**, then take all of the remaining rows and place them into a list.

In [9]:
df_1['expanded_urls'] = df_1.expanded_urls.fillna(value='https://www.twitter.com')
urls = df_1.expanded_urls
urls = list(urls)

Next I created a new empty list, **status**, and used a loop to extract the id's from **urls**, appending
each id to **status**. As a conditional, also appended the string '0' to status, every time my search
was unable to find an accurate tweet_id

In [10]:
status= []
for i in range(len(urls)): 
    t= urls[i]
    if bool(re.search(r'\d', t)) == True:
        twt = re.findall(r'\d+', t)[0]
        status.append(twt)
    else:
        status.append('0')
status[:5]

['892420643555336193',
 '892177421306343426',
 '891815181378084864',
 '891689557279858688',
 '891327558926688256']

I replaced my DataFrame column **tweet_id** with the values in **status** then removed all the values
that contained the string '0'.

In [11]:
df_1['tweet_id'] = status
df_1 = df_1.query("tweet_id != '0'")

My last step was to change the data type of **tweet_id** from strings to the int64 data type, standardizing the **tweet_id** data type accross various sources of data to be able to merge in the future, and preparing them for my next gathering step of the Twitter API. Additionally, int64 also is the most efficient way to store values of unique digits, as integers require much less memory than a string.

In [12]:
df_1 = df_1.astype({'tweet_id':'int64'})

In [13]:
tweet_ids = df_1['tweet_id']

In [14]:
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
#count = 0
#fails_dict = {}
#start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
#with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#    for tweet_id in tweet_ids:
#        count += 1
#        print(str(count) + ": " + str(tweet_id))
#        try:
#            tweet = api.get_status(tweet_id, tweet_mode='extended')
#            print("Success")
#            json.dump(tweet._json, outfile)
#            outfile.write('\n')
#        except tweepy.TweepError as e:
#            print("Fail")
#            fails_dict[tweet_id] = e
            
#            pass
#end = timer()
#print(end - start)
#print(fails_dict)


In [15]:
#df_2 = pd.DataFrame(columns=['tweet_id','retweet count','favorite_count'])

#with open('tweet_json.txt') as i:
#    for line in i:
#        stat = json.loads(line)
#        tweet_id = stat['id_str']
#        retweet_count = stat['retweet_count']
#        favorite_count = stat['favorite_count']
#        df_2 = df_2.append(pd.DataFrame([[tweet_id,retweet_count,
#                                         favorite_count]], 
#                                       columns=['tweet_id',
#                                        'retweet_count', 'favorite_count']))
#df_2 = df_2.reset_index(drop=True)
#df_2.head()
        

In [16]:
#df_2.to_csv (r'C:\Users\tssan\Desktop\Udacity Projects\4wrangle\WeRateDogs.csv',
#                          index = None, header=True, sep='\t', encoding='utf-8')

In [17]:
df_2 = pd.read_csv('WeRateDogs.csv', sep='\t',encoding='utf-8')

# Assessment
### Visual Assessment

In [18]:
df_1.loc[:9].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,nan,nan,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,nan,nan,nan,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,nan,nan,2017-08-01 00:17:27 +0000,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",nan,nan,nan,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,nan,nan,2017-07-31 00:18:03 +0000,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,nan,nan,nan,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,nan,nan,2017-07-30 15:58:51 +0000,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,nan,nan,nan,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,nan,nan,2017-07-29 16:00:24 +0000,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",nan,nan,nan,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None
5,891087950875897856,nan,nan,2017-07-29 00:08:17 +0000,Twitter for iPhone,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,nan,nan,nan,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
6,890971913173991426,nan,nan,2017-07-28 16:27:12 +0000,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below https://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,nan,nan,nan,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None
7,890729181411237888,nan,nan,2017-07-28 00:22:40 +0000,Twitter for iPhone,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,nan,nan,nan,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
8,890609185150312448,nan,nan,2017-07-27 16:25:51 +0000,Twitter for iPhone,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,nan,nan,nan,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None
9,890240255349198849,nan,nan,2017-07-26 15:59:51 +0000,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,nan,nan,nan,https://twitter.com/dog_rates/status/890240255349198849/photo/1,14,10,Cassie,doggo,None,None,None


- Five columns in the **df_1** Dataframe appear to contain missing values: 
    - **in_reply_to_status_id**, 
    - **in_reply_to_user_id**, 
    - **retweeed_status_id**, 
    - **retweeted_status_user_id**,  
    - **retweeted_status_timestamp** 
 
- The **text** column contains the body of the tweet, but then ends each tweet with a hyperlink.  The hyperlink is already represented in the **expanded_urls** column, so having it shown twice is redundant information and is not really part of the text of the tweet. 
- The columns **doggo**, **floofer**, **pupper**, and **puppo** have the string "None" in many of there rows. These columns seem to be categorical varibales and would benefit by finding a melt them into a single categorical column.
- The **timestamp** column is formatted down to the millisecond. This is accurate and thourough information, but for future timeseries analysis, I will convert the time in a date format of yyyy-MM-dd instead of the yyyy-MM-dd HH:mm:ss. SSS it is in now.

In [19]:
images.loc[:10].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.0614285,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.0741917,True,Rhodesian_ridgeback,0.07201,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.0161992,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.0458854,False,terrapin,0.0178853,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.0582794,True,fur_coat,0.0544486,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.0145938,False,golden_retriever,0.00795896,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.0820861,True


- The columns **p1**, **p2**, and **p3** are all in snake case format and some of the categorical values are capitalized while others are not.  To clean this up I would like to remove the underscore(s) of each row and put the classications in title format 

In [20]:
df_2.loc[:9].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,favorite_count,retweet count,retweet_count,tweet_id
0,37135,nan,7937,892420643555336193
1,31939,nan,5900,892177421306343426
2,24054,nan,3899,891815181378084864
3,40456,nan,8095,891689557279858688
4,38672,nan,8778,891327558926688256
5,19442,nan,2917,891087950875897856
6,11340,nan,1937,890971913173991426
7,62632,nan,17701,890729181411237888
8,26740,nan,4020,890609185150312448
9,30632,nan,6903,890240255349198849


**retweet count** is an inconsistent format by not having an underscore to replace the space between words. By grouping column names in snake case the column names of a dataframe are callable. This would likely be a column name that I renamed to much python convention for a column name but it looks like it is a duplicate column to **reteet_count** and has a majority of NaN values. I will drop the column **retweet count** to tidy up the **df_2** DataFrame.

### Programatic Assessment

In [21]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2270 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2270 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2270 non-null object
source                        2270 non-null object
text                          2270 non-null object
retweeted_status_id           176 non-null float64
retweeted_status_user_id      176 non-null float64
retweeted_status_timestamp    176 non-null object
expanded_urls                 2270 non-null object
rating_numerator              2270 non-null int64
rating_denominator            2270 non-null int64
name                          2270 non-null object
doggo                         2270 non-null object
floofer                       2270 non-null object
pupper                        2270 non-null object
puppo                         2270 non-null object
dtypes: float64(4), int64(3), ob

- As observed above, five columns have missing values.
    - **in_reply_to_status_id** and **in_reply_to_user_id** each have 2247 null values
    - **retweeed_status_id**, **retweeted_status_user_id**, and **retweeted_status_timestamp** each have 2094 null values

In [22]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166 entries, 0 to 2165
Data columns (total 4 columns):
favorite_count    2166 non-null int64
retweet count     0 non-null float64
retweet_count     2166 non-null float64
tweet_id          2166 non-null int64
dtypes: float64(2), int64(2)
memory usage: 67.8 KB


- After running a summary of the dataframe **df_2** we see that all 2166 values are missing in the column **retweet count**

In [23]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


- The **images** dataframe has no missing values. 

In [24]:
a = df_1.doggo.value_counts()
b = df_1.floofer.value_counts()
c = df_1.pupper.value_counts()
d = df_1.puppo.value_counts()

print(f'{a},\n\n{b},\n\n{c},\n\n{d}')

None     2176
doggo      94
Name: doggo, dtype: int64,

None       2260
floofer      10
Name: floofer, dtype: int64,

None      2017
pupper     253
Name: pupper, dtype: int64,

None     2241
puppo      29
Name: puppo, dtype: int64


In [25]:
df_1.rating_numerator.describe()

count    2270.000000
mean       12.238767
std        38.918696
min         0.000000
25%        10.000000
50%        11.000000
75%        12.000000
max      1776.000000
Name: rating_numerator, dtype: float64

In [26]:
df_1.rating_numerator.value_counts().sort_index()

0         2
1         6
2         9
3        19
4        16
5        35
6        32
7        53
8       102
9       155
10      451
11      448
12      537
13      335
14       49
15        1
24        1
26        1
27        1
44        1
45        1
50        1
60        1
75        2
80        1
84        1
88        1
99        1
121       1
143       1
144       1
165       1
204       1
420       1
1776      1
Name: rating_numerator, dtype: int64

For most of the cases if a numerator had a value
greater than 14 it was only used once, however 75 occurs twice.
I decided to look into this a little further to figure out why this was.

In [27]:
ind1, ind2 = df_1.text[df_1.rating_numerator== 75].index.tolist()
num = 75
text1, text2 = df_1.text[ind1], df_1.text[ind2]
print(f'Index: {ind1}\nNumerator: {num}\n{text1} \n\nIndex: {ind2}\nNumerator: {num}\n{text2}')

Index: 340
Numerator: 75
RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu… 

Index: 695
Numerator: 75
This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS


By taking a closer look at these two tweets, I found two problems. The first is that the numerator is not 75, as it is listed in the df_1 dataset, its actually 10. The number 75 was misrepresented as the numerator in error when a rating of 9.75(a sneaky nod to the secret entrance of the Hogwarts Express on platform 9 and 3/4) was given to Logan and second, the indexed tweet #340 happens to be a duplicate of the indexed tweet #695 

In [28]:
df_1.tweet_id.duplicated().any()

True

Confirming what we already knew from the previous cell, there are duplicates in the **df_1** dataset that will need to be dropped.

In [29]:
df_2.tweet_id.duplicated().any()

True

In [30]:
images.tweet_id.duplicated().any()

False

Like the **df_1** dataset, duplicates were found in **df_2** as well. These duplicates will also need to be dropped. Luckily, no duplicates were found in the **images** dataset. 

In [31]:
df_1.rating_denominator.describe()

count    2270.000000
mean       10.467841
std         6.863241
min         2.000000
25%        10.000000
50%        10.000000
75%        10.000000
max       170.000000
Name: rating_denominator, dtype: float64

In [32]:
df_1.rating_denominator.value_counts().sort_index()

2         1
7         1
10     2251
11        3
20        1
40        1
50        3
70        1
80        2
90        1
110       1
120       1
130       1
150       1
170       1
Name: rating_denominator, dtype: int64

In [33]:
df_1.query('rating_denominator > 10').rating_denominator.count()

17

In [34]:
df_1.query('rating_denominator < 10').rating_denominator.count()

2

2251 of the 2270 observations have a denominator of ten. I decided to look into this further to see if I could find out why this was. 
- 2 tweets have a denominator less than ten
- 17 tweets have denominators greater than ten. 


In [35]:
ind1, ind2 = df_1.text[df_1.rating_denominator < 10].index.tolist()
denom1, denom2 = df_1.rating_denominator[ind1], df_1.rating_denominator[ind2]
text1, text2 = df_1.text[ind1], df_1.text[ind2]
print(f'Index: {ind1}\nDenominator:{denom1}\n{text1} \n\nIndex: {ind2} \nDenominator:{denom2}\n{text2}')

Index: 516
Denominator:7
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx 

Index: 2335 
Denominator:2
This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv


For the the denominators that had values less then ten
- Tweet indexed #516 was one few times where WeRateDogs was not given a rating.
- Tweet indexed #2335 misrepresented the rating in error when a fraction of 1/2 was included in the text.

In [36]:
ind = df_1.query('rating_denominator > 10')['rating_denominator'].index.tolist()
denom = df_1.query('rating_denominator > 10')['rating_denominator'].tolist()
text = df_1.query('rating_denominator > 10')['text'].tolist()
for i in range(len(ind)):
    nex_ind = ind[i]
    nex_denom = denom[i]
    nex_text = text[i]
    print(f'Index: {nex_ind}\nDenominator: {nex_denom}\n{nex_text}\n')

Index: 433
Denominator: 70
The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd

Index: 784
Denominator: 11
RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…

Index: 902
Denominator: 150
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE

Index: 1068
Denominator: 11
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ

Index: 1120
Denominator: 170
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv

Index: 1165
Denominator: 20
Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a

Index: 1202
Denominator: 50
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq

Index: 1228
Denominator: 90
Happ

In [37]:
df_1['timestamp'].min(), df_1['timestamp'].max()

('2015-11-15 22:32:08 +0000', '2017-08-01 16:23:56 +0000')

In [38]:
df_1.timestamp[:5]

0    2017-08-01 16:23:56 +0000
1    2017-08-01 00:17:27 +0000
2    2017-07-31 00:18:03 +0000
3    2017-07-30 15:58:51 +0000
4    2017-07-29 16:00:24 +0000
Name: timestamp, dtype: object

### Summary of Assessment

#### Quality issues
**df_1** dataset
- stuff
- stuff
- stuff
- stuff
- stuff

**df_2** dataset
- stuff
- stuff
- stuff

**images** dataset
- No quality errors were found in this dataset


#### Tidiness Issues
**df_1** dataset 
- stuff
- stuff
- stuff

**df_2** dataset
- stuff
- stuff
- stuff

**images** dataset
- stuff
- stuff



# Clean

In [39]:
df_1_clean = df_1.copy()
df_2_clean = df_2.copy()
images_clean = images.copy()

### Quality 

##### Define
- 1. Dropping the columns with a majority of NaN values from the **df_1** dataframe

##### Code

In [40]:
df_1.drop(columns=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id',
                   'retweeted_status_user_id','retweeted_status_timestamp'
                   ], inplace=True)

##### Test the results

In [41]:
df_1.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

##### Define
- 2. Melting Doggo, Pupper, Floofer, and Puppo down to one category variable named Cute_Name in the **df_1** Dataframe  

#####  Code

In [42]:
a = df_1['doggo']
b = df_1['pupper']
c = df_1['floofer']
d = df_1['puppo']


df_1['cute_name'] = a.str.cat(b.replace('None','')).str.cat(c.replace('None','')).str.cat(d.replace('None',''))

In [43]:
dict_dog = {'Nonefloofer':'floofer','Nonepupper':'pupper','Nonepuppo':'puppo','doggopupper':'doggo, pupper',
           'doggofloofer':'doggo, floofer','doggopuppo':'doggo, puppo','None':'Other'}

for key, value in dict_dog.items():
    df_1['cute_name'] = df_1.cute_name.replace(key,value)
    
df_1.drop(columns=['doggo', 'floofer','pupper', 'puppo'],inplace=True)

##### Test the results

In [44]:
df_1.cute_name.value_counts()

Other             1898
pupper             241
doggo               80
puppo               28
doggo, pupper       12
floofer              9
doggo, puppo         1
doggo, floofer       1
Name: cute_name, dtype: int64

In [45]:
df_1.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'cute_name'],
      dtype='object')

##### Define
- 3.  Fixing Tweet 516 (No rating given) in the **df_1** Dataframe

#####  Code

In [46]:
df_1.loc[516, 'rating_numerator']= np.median(df_1.rating_numerator)
df_1.loc[516, 'rating_denominator']= np.median(df_1.rating_denominator)

##### Test the results

In [47]:
print(f"Since Sam was never given a rating, I gave him the median rating which is {df_1.loc[516,'rating_numerator']}/{df_1.loc[516,'rating_denominator']}")

Since Sam was never given a rating, I gave him the median rating which is 11.0/10.0


##### Define
- 4.Dropping the duplicate in the **df_1** and **df_2** Dataframes

#####  Code

In [48]:
df_1 = df_1.drop_duplicates(subset=['tweet_id'],keep='first').reset_index()
df_2 = df_2.drop_duplicates(subset=['tweet_id'],keep='first').reset_index()


_4. Test the results_

In [49]:
df_1.tweet_id.duplicated().any(), df_2.tweet_id.duplicated().any()

(False, False)

##### Define
- 5. Dropping columns with null data in the **df_2** Dataframe

#####  Code

In [50]:
df_2.drop(columns='retweet count', inplace= True)

##### Test the results

In [51]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 4 columns):
index             2026 non-null int64
favorite_count    2026 non-null int64
retweet_count     2026 non-null float64
tweet_id          2026 non-null int64
dtypes: float64(1), int64(3)
memory usage: 63.4 KB


##### Define
- 6. Dropping the 'source' column from **df** table

#####  Code

In [52]:
df_1.drop(columns=['source'],inplace=True)

##### Test the results

In [53]:
df_1.columns

Index(['index', 'tweet_id', 'timestamp', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'cute_name'],
      dtype='object')

##### Define
- 8. 

#####  Code

In [54]:
indices = []
for i, row in df_1.iterrows():
    if df_1.rating_denominator[i] != 10:
        in_str= row['text']
        try:
            test = re.search(r'\d+(?:\.\d+)?/10', in_str).group()
            den = re.findall(r'\d+', test)[1]
            test = re.findall(r'\d+', test)[0]
            df_1.at[i,'rating_numerator'] = test
            df_1.at[i,'rating_denominator'] = den
            indices.append(i)
        except:
            test = 'Unchanged'
        if test == 'Unchanged':
            denominator = 'Unchanged'
        else:
            denominator = den
                
        print(f'Index: {i}\nNumerator: {test}\nDenominator: {denominator}\n{in_str}\n')
        
    else:
        pass

Index: 377
Numerator: Unchanged
Denominator: Unchanged
The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd

Index: 655
Numerator: 14
Denominator: 10
RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…

Index: 750
Numerator: Unchanged
Denominator: Unchanged
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE

Index: 934
Numerator: Unchanged
Denominator: Unchanged
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv

Index: 977
Numerator: 13
Denominator: 10
Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a

Index: 1011
Numerator: 11
Denominator: 10
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq

Index: 1032
Numerator: Unchanged
Denominator: Unchanged
Happy Saturday here's 9 

##### Test the results

In [55]:
for i in range(len(indices)):
    ind = indices[i]
    num = df_1.rating_numerator[indices[i]]
    den = df_1.rating_denominator[indices[i]]
    print(f'Tweet #{ind} has a rating of {num}/{den}')

Tweet #655 has a rating of 14.0/10.0
Tweet #977 has a rating of 13.0/10.0
Tweet #1011 has a rating of 11.0/10.0
Tweet #1406 has a rating of 10.0/10.0
Tweet #2033 has a rating of 9.0/10.0


##### Define
- 9. Normalizing Denominators to a Standard of 10 structuring data to facilitate analysis **df_1**

#####  Code

In [56]:
norm_numerator= []
norm_denominator= []
for i, row in df_1.iterrows():
    norm = 10 / row['rating_denominator']
    num = row['rating_numerator'] = row['rating_numerator']* norm
    norm_numerator.append(num)
    den = row['rating_denominator'] = row['rating_denominator']* norm
    norm_denominator.append(den)
    

In [57]:
df_1['rating_numerator'] = norm_numerator
df_1['rating_denominator'] = norm_denominator

##### Test the results

In [58]:
df_1.query('rating_denominator !=10')['rating_denominator'].any()

False

In [59]:
df_1.drop(columns=['rating_denominator'],inplace = True)
df_1.rename(columns={"rating_numerator": "rating"},inplace=True)

In [60]:
df_1.columns

Index(['index', 'tweet_id', 'timestamp', 'text', 'expanded_urls', 'rating',
       'name', 'cute_name'],
      dtype='object')

In [61]:
df_1.rating.value_counts()

12.0      468
10.0      428
11.0      408
13.0      284
9.0       152
8.0        96
7.0        51
14.0       44
5.0        35
6.0        32
3.0        19
4.0        15
2.0         9
1.0         4
0.0         2
27.0        1
1776.0      1
15.0        1
26.0        1
420.0       1
75.0        1
Name: rating, dtype: int64

##### Define
- 12.  Correcting Ratings Containing Decimal Places, Using Regular Expression to loop through each text and pull out any digits with decimal places in **df**

#####  Code

In [62]:
for i, row in df_1.iterrows():
    in_str= row['text']
    split = re.split(r'/' , in_str)[0]
    if bool(re.search(r'\d+\.\d+$', split)) == True:
        after = re.findall(r'\d+\.\d+', split)[0]
        before = df_1.loc[i, 'rating']
        df_1.loc[i, 'rating'] = after
        print(f"The rating in row {i} used to be {before} and now is {after}.")
    else:
        pass

The rating in row 43 used to be 5.0 and now is 13.5.
The rating in row 301 used to be 75.0 and now is 9.75.
The rating in row 638 used to be 27.0 and now is 11.27.
The rating in row 1450 used to be 26.0 and now is 11.26.


In [63]:
df_1.rating.dtype

dtype('O')

In [64]:
#No longer need the denominator columns, and the rating_numerator column
#can just become rating (on a standardized scale of 10)
df_1['rating'] = df_1['rating'].astype('float64')


##### Test the results

In [65]:
check = df_1.rating
text = df_1.text
divide = "---" * 38

for i in range(len(check)):
    chk_dec = check[i]
    txt = text[i] 
    if bool(chk_dec.is_integer()) == False:
        print(f'Now row {i} has the correct rating of {chk_dec}. Take a look. \n\n{txt}\n{divide}\n')
    else:
        pass

Now row 43 has the correct rating of 13.5. Take a look. 

This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948
------------------------------------------------------------------------------------------------------------------

Now row 301 has the correct rating of 9.75. Take a look. 

RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…
------------------------------------------------------------------------------------------------------------------

Now row 638 has the correct rating of 11.27. Take a look. 

This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq
------------------------------------------------------------------------------------------------------------------

Now row 1450 has the correct rating

##### Define
- 13. 

#####  Code

In [66]:
# Rounding Ratings to get a discrete scale
for i, row in df_1.iterrows():
    before = df_1.loc[i, 'rating']
    after = round(df_1.rating[i], 0)
    df_1.loc[i, 'rating'] = after
    if bool(before == after) == False :
        print(f"The rating in row {i} used to be {before} and now it is {after}.")
    else:
        pass

The rating in row 43 used to be 13.5 and now it is 14.0.
The rating in row 301 used to be 9.75 and now it is 10.0.
The rating in row 638 used to be 11.27 and now it is 11.0.
The rating in row 1450 used to be 11.26 and now it is 11.0.


In [67]:
df_1.rating.dtype

dtype('float64')

##### Define
- 14. 


In [68]:
#####  Code

In [75]:
df_1.rating = df_1.rating.astype(int) 

In [ ]:
##### Test the results


In [76]:
df_1.rating.dtype

dtype('int32')

#####  Code

In [ ]:
df_1['text'] = df_1.text.apply(lambda text: text.split('http')[0])

##### Test the results

In [ ]:
line1 = df_1.text[np.random.randint(0,10)]
line2 = df_1.text[np.random.randint(0,10)]
line3 = df_1.text[np.random.randint(0,10)]

print(f'{line1}\n\n{line2} \n\n{line3}')

### Tidiness

##### Define
- 16. Merging **df_1**, **df_2**, and **images** Dataframes into a single table

#####  Code

In [ ]:
df = df_1.merge(df_2, on='tweet_id', how='inner')

In [ ]:
df = df.merge(images, on='tweet_id', how='inner')
df = df.reset_index(drop=True)

In [ ]:
##### Test the results

In [ ]:
df.columns

In [ ]:
df.info()

##### Define
- 16. 

#####  Code

In [ ]:
df = df.astype({'rating' : int, 'retweet_count' : int} )

##### Test the results

In [ ]:
df.info()

Define
15.

In [ ]:
time = df['timestamp']
time = list(time)

In [ ]:
times = []
for i in range(len(time)): 
    t= time[i]
    t = t.rpartition(' ')[0]
    stamp = t.rpartition(' ')[0]
    times.append(stamp)

In [ ]:
df['timestamp'] = times
df['timestamp'] = pd.to_datetime(df['timestamp'])

##### Test the results

In [ ]:
df.timestamp[:5]

In [ ]:
#df.to_csv (r'~/Desktop/Udacity Projects/4wrangle/WeRateDogs_Clean.csv',
                        #index = False, header=True, sep='\t', encoding='utf-8')

In [ ]:
df_cln = df.copy()

In [ ]:
#df_cln = pd.read_csv('WeRateDogs_Clean.csv', sep='\t',encoding='utf-8')

# Finding a Winner

In [ ]:
df_cln.groupby(['rating']).count()

In [ ]:
df_cln[df_cln['rating'] == 420]

In [ ]:
df_cln.text[1712]

In [ ]:
print(df_cln.jpg_url[1712])

### At second place Snoop Dogg comes in at very impressive 420 out of 10
<img src="second.jpg"/>

In [ ]:
df_cln[df_cln['rating'] == 1776]

In [ ]:
df_cln.text[757]

In [ ]:
print(df_cln.jpg_url[757])

### Congratulation to the winner coming in at 1776 out of 10, such a good boy!
<img src="winner.jpg"/>

## Distribution of Dog Ratings by Dog Types

- The two winners above are data points that were accurately gathered, but their ratings are also global outliers to the rest of the observations. But as we can see from the box plots below, these global outliers greatly increase the standard deviation of the variable **rating** in our DataFrame.  
- To simplify our analysis, I decided to remove the two outliers, and take a deeper look into the distribution of the ratings by dog type.

In [ ]:
df_before= df_cln.copy()
df_after = df_cln[df_cln['rating'] < 15]

#df_before= plots of the distribution of the data before and after removing the two outliers
f, axes = plt.subplots(1, 2, figsize=(12,4))

sns.boxplot(data=df_before,x= 'rating',orient='w', ax=axes[0])
axes[0].set_title('Before, Including Global Outliers')


sns.boxplot(data=df_after, x= 'rating',orient='w', ax=axes[1])
axes[1].set_title('After, Excluding Global Outliers');

In [ ]:
df_rate = df_after.copy()

In [ ]:
df_rate['p1'].value_counts()

In [ ]:
dog_types = df_rate['p1'].value_counts().keys()
dog_types = list(dog_types)

In [ ]:
dog_types = dog_types[:9]

In [ ]:
for d in dog_types:
    df_other = df_rate[(df_rate.p1 != d)]

df_other = df_other.rating.value_counts().sort_index()
df_other = df_other.rename_axis('rating').to_frame('other')

for d in dog_types:
    df_add = df_rate[df_rate['p1'] == d ]
    df_add = df_add.rating.value_counts().sort_index()
    df_add = df_add.rename_axis('rating').to_frame(d)
    df_other = df_other.merge(df_add, on='rating',how='left').fillna(0)
    

df_other.loc[:,'Total'] = df_other.sum(axis=1)
df_other.index = df_other.index.map(str)
df_other.columns = [x.lower() for x in df_other.columns]
df_all = df_other.copy()
df_all

In [ ]:
b='Black'
r = df_all.index
t = df_all.total
c= sns.color_palette()[9]
f_sz=12
w='whitesmoke'
fig = plt.figure(figsize=(6,7), facecolor=c)

            
ax = plt.subplot(1, 1, 1)
plt.barh(range(len(r)),t, color=w)
plt.yticks(range(len(r)),r,fontsize=12, color=b)
plt.ylabel('Rating',color=w,fontsize=f_sz)
plt.xticks(color=b,fontsize=f_sz)
plt.xlabel('Frequency',color=w,fontsize=f_sz)
plt.title('Total Distribution of Dog Ratings',color='Black',fontsize=22)

ax.set_axisbelow(True)
ax.yaxis.grid(color='white', linestyle='dashed')
plt.box(False)
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import Grid
i = df_all.index
t = df_all.total
r = df_all.index
ch = df_all.chihuahua
g = df_all.golden_retriever
l = df_all.labrador_retriever
p = df_all.pembroke
pu = df_all.pug
ow = df_all.chow
s = df_all.samoyed
pm = df_all.pomeranian
tp = df_all.toy_poodle
c= sns.color_palette()[9]
w='whitesmoke'
b='Black'
f_sz=16
lim = [0,40]
fig = plt.figure(figsize=(10,14), facecolor=c)
grid = Grid(fig, rect=111, nrows_ncols=(2,2),
            axes_pad=0.25, label_mode='L')



ax1= plt.subplot(3, 3, 1)
plt.barh(range(len(r)),g, color=w)
plt.yticks(range(len(r)),r,fontsize=11,color=b)
plt.xticks(color=b,fontsize=11)
plt.xlabel('Frequency',color=w)
plt.ylabel('Rating',color=w)
plt.box(False)
plt.title('Golden Retriever',fontsize=f_sz,color=b)
plt.xlim(lim)
ax1.set_axisbelow(True)
ax1.yaxis.grid(color='white', linestyle='dashed')

ax2= plt.subplot(3, 3, 2)
plt.barh(range(len(r)),l, color=w)
plt.yticks(range(len(r)),r)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Labrador',fontsize=f_sz,color=b)
plt.xlim(lim)
ax2.set_axisbelow(True)
ax2.yaxis.grid(color='white', linestyle='dashed')

ax3 = plt.subplot(3, 3, 3)
plt.barh(range(len(r)),ch, color=w)
plt.yticks(range(len(r)),r)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax3.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Chihuah',fontsize=f_sz,color=b)
plt.xlim(lim)
ax3.set_axisbelow(True)
ax3.yaxis.grid(color='white', linestyle='dashed')

ax4 = plt.subplot(3, 3, 4)
plt.barh(range(len(r)),p, color=w)
plt.yticks(range(len(r)),r,fontsize=11,color=b)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.ylabel('Rating',color=w)
plt.box(False)
plt.title('Pembroke',fontsize=f_sz,color=b)
plt.xlim(lim)
ax4.set_axisbelow(True)
ax4.yaxis.grid(color='white', linestyle='dashed')

ax5 = plt.subplot(3, 3, 5)
plt.barh(range(len(r)),pu, color=w)
plt.yticks(range(len(r)),r)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax5.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Pug', fontsize=f_sz, color=b)
plt.xlim(lim)
ax5.set_axisbelow(True)
ax5.yaxis.grid(color='white', linestyle='dashed')

ax6 = plt.subplot(3, 3, 6)
plt.barh(range(len(r)),ow, color=w)
plt.yticks(range(len(r)),r)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax6.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Chow', fontsize=f_sz, color=b)
plt.xlim(lim)
ax6.set_axisbelow(True)
ax6.yaxis.grid(color='white', linestyle='dashed')

ax7 = plt.subplot(3, 3, 7)
plt.barh(range(len(r)),ow, color=w)
plt.yticks(range(len(r)),r,fontsize=11,color=b)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.ylabel('Rating',color=w)
plt.setp(ax7.get_yticklabels(), visible=True)
plt.box(False)
plt.title('Samoyed', fontsize=f_sz, color=b)
plt.xlim(lim)
ax7.set_axisbelow(True)
ax7.yaxis.grid(color='white', linestyle='dashed')

ax8 = plt.subplot(3, 3, 8)
plt.barh(range(len(r)),pm, color=w)
plt.yticks(range(len(r)),r,)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax8.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Pomeranian', fontsize=f_sz, color=b)
plt.xlim(lim)
ax8.set_axisbelow(True)
ax8.yaxis.grid(color='white', linestyle='dashed')

ax9 = plt.subplot(3, 3, 9)
plt.barh(range(len(r)),tp, color=w)
plt.yticks(range(len(r)),r,)
plt.xticks(color=b, fontsize=11)
plt.xlabel('Frequency',color=w)
plt.setp(ax9.get_yticklabels(), visible=False)
plt.box(False)
plt.title('Toy Poodle', fontsize=f_sz, color=b)
plt.xlim(lim)
ax9.set_axisbelow(True)
ax9.yaxis.grid(color='white', linestyle='dashed')

plt.tight_layout()
plt.show()

## Time Series of WeRateDogs Tweets

In [ ]:
df_time = df_cln.copy()

In [ ]:
# TimeSeries of Twitter Activity 

#Favorite and Retweet series
time_faves = pd.Series(data=df_time['favorite_count'].values, index=df_time['timestamp']).sort_index(ascending=True)
time_retweets = pd.Series(data=df_time['retweet_count'].values, index=df_time['timestamp']).sort_index(ascending=True)

#Plotting both series to the same axis
size= (12,8)

time_faves.plot(kind='area',figsize=size, label='favorites',
                color='deepskyblue',legend=True)
time_retweets.plot(kind='area',figsize=size, label='retweet',
                   color='crimson',legend=True)

#Labels and preferences for the visualization

plt.title('WeRateDogs Time Series of Favorites and Retweets',fontsize=18)
plt.xlabel('Date',fontsize=18,color='crimson') 
plt.ylabel('Twitter Activity Count',fontsize=18, color='crimson')
plt.xticks(rotation=45)
plt.legend(fontsize=14,frameon=False)


plt.box(False)
plt.show();